# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [4]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [12]:
# Supprimer les doublons
df_youtube = df_youtube.drop_duplicates()

# Gérer les valeurs manquantes
df_youtube = df_youtube.fillna('Unknown')  # ou df_youtube.dropna()

df_youtube.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7985 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        7985 non-null   object
 1   title           7985 non-null   object
 2   channel_title   7985 non-null   object
 3   category_id     7985 non-null   int64 
 4   tags            7985 non-null   object
 5   views           7985 non-null   int64 
 6   likes           7985 non-null   int64 
 7   dislikes        7985 non-null   int64 
 8   comment_total   7985 non-null   int64 
 9   thumbnail_link  7985 non-null   object
 10  date            7985 non-null   object
dtypes: int64(5), object(6)
memory usage: 748.6+ KB


### Importer les données

In [13]:
a = df_youtube.to_dict(orient='records')
collection.drop()
collection.insert_many(a)


InsertManyResult([ObjectId('674f3143ea2eb205aa009518'), ObjectId('674f3143ea2eb205aa009519'), ObjectId('674f3143ea2eb205aa00951a'), ObjectId('674f3143ea2eb205aa00951b'), ObjectId('674f3143ea2eb205aa00951c'), ObjectId('674f3143ea2eb205aa00951d'), ObjectId('674f3143ea2eb205aa00951e'), ObjectId('674f3143ea2eb205aa00951f'), ObjectId('674f3143ea2eb205aa009520'), ObjectId('674f3143ea2eb205aa009521'), ObjectId('674f3143ea2eb205aa009522'), ObjectId('674f3143ea2eb205aa009523'), ObjectId('674f3143ea2eb205aa009524'), ObjectId('674f3143ea2eb205aa009525'), ObjectId('674f3143ea2eb205aa009526'), ObjectId('674f3143ea2eb205aa009527'), ObjectId('674f3143ea2eb205aa009528'), ObjectId('674f3143ea2eb205aa009529'), ObjectId('674f3143ea2eb205aa00952a'), ObjectId('674f3143ea2eb205aa00952b'), ObjectId('674f3143ea2eb205aa00952c'), ObjectId('674f3143ea2eb205aa00952d'), ObjectId('674f3143ea2eb205aa00952e'), ObjectId('674f3143ea2eb205aa00952f'), ObjectId('674f3143ea2eb205aa009530'), ObjectId('674f3143ea2eb205aa0095

## Question 1  

In [16]:
#- 1) Récupérer toutes les vidéos de la chaîne Apple.
cur = collection.find({"channel_title": "Apple"})

for i in cur:
    print(i)

{'_id': ObjectId('674f3143ea2eb205aa009519'), 'video_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple,iPhone 10,iPhone Ten,iPhone,Portrait Lighting,A11 Bionic,augmented reality,emoji,animoji,Face ID,Apple Pay,camera,smartphone', 'views': 7860119, 'likes': 185853, 'dislikes': 26679, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 'Unknown'}
{'_id': ObjectId('674f3143ea2eb205aa0095e3'), 'video_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple,iPhone 10,iPhone Ten,iPhone,Portrait Lighting,A11 Bionic,augmented reality,emoji,animoji,Face ID,Apple Pay,camera,smartphone', 'views': 12200526, 'likes': 258842, 'dislikes': 44339, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': 'Unknown'}
{'_id': ObjectId('674f3143ea2eb205aa0096bb'), 'video_

## Question 2

In [17]:
#- 2) Compter le nombre de catégories différentes

pipeline = [
    
    {"$group": {"_id": "$category_id"}},
    {"$count": "categories"}
]

cur = collection.aggregate(pipeline)

print(list(cur))

[{'categories': 16}]


## Question 3

In [26]:
#- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.

cur = collection.find()

for elem in cur:

    tags_list = elem['tags'] #.split('|')
    for tag in tags_list:
        if tag == '[none]':
            tags_list.remove(tag)
    collection.update_one(
            {'_id': elem['_id']},
            {'$set': {'tags': tags_list}}
        )


## Question 4

In [31]:
#- 4) Récupérer les vidéos les plus vues.
cur = collection.find().sort("views", -1).limit(5)

for i in cur:
    print(i)





{'_id': ObjectId('674f3143ea2eb205aa009c8b'), 'video_id': 'MBdVXkSdhwU', 'title': "BTS (방탄소년단) 'DNA' Official MV", 'channel_title': 'ibighit', 'category_id': 10, 'tags': ['BIGHIT,빅히트,방탄소년단,BTS,BANGTAN,방탄'], 'views': 41500672, 'likes': 2010366, 'dislikes': 78076, 'comment_total': 736179, 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg', 'date': 'Unknown'}
{'_id': ObjectId('674f3143ea2eb205aa009bb6'), 'video_id': 'MBdVXkSdhwU', 'title': "BTS (방탄소년단) 'DNA' Official MV", 'channel_title': 'ibighit', 'category_id': 10, 'tags': ['BIGHIT,빅히트,방탄소년단,BTS,BANGTAN,방탄'], 'views': 38013692, 'likes': 1950475, 'dislikes': 75294, 'comment_total': 713293, 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg', 'date': 'Unknown'}
{'_id': ObjectId('674f3143ea2eb205aa0095c9'), 'video_id': 'tt2k8PGm-TI', 'title': 'ZAYN - Dusk Till Dawn ft. Sia', 'channel_title': 'ZaynVEVO', 'category_id': 10, 'tags': ['Dusk Till Dawn,Pop,RCA Records Label,ZAYN feat. Sia'], 'views': 36323498, 'li

## Question 5

In [32]:
#- 5) Compter le nombre moyen de vues en fonction de la catégorie.

pipeline = [

    {"$group": {"_id": "$category_id", "average_views": {"$avg": "$views"}}}
]

cur = collection.aggregate(pipeline)

for elem in cur:
    print(elem)

{'_id': 23, 'average_views': 1240073.294039735}
{'_id': 25, 'average_views': 540955.5569823436}
{'_id': 17, 'average_views': 728434.3243902439}
{'_id': 19, 'average_views': 464041.0833333333}
{'_id': 20, 'average_views': 681081.8170731707}
{'_id': 43, 'average_views': 8492.5}
{'_id': 2, 'average_views': 607693.8706896552}
{'_id': 10, 'average_views': 1180330.6701444623}
{'_id': 1, 'average_views': 1047788.4373333333}
{'_id': 22, 'average_views': 971532.7573696146}
{'_id': 28, 'average_views': 924730.548828125}
{'_id': 26, 'average_views': 537665.1806674339}
{'_id': 29, 'average_views': 1110334.2142857143}
{'_id': 27, 'average_views': 547582.4700598803}
{'_id': 15, 'average_views': 651404.3879310344}
{'_id': 24, 'average_views': 1154868.4397251718}


## Question 6 

In [38]:
#- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

pipeline = [

    {"$group": {"_id": "$channel_title", "average_likes": {"$avg": "$likes"}}},
    {"$sort": {"average_likes": -1}}
]

cur = collection.aggregate(pipeline)

for elem in cur:
    print(elem)

{'_id': 'ZaynVEVO', 'average_likes': 1431683.0}
{'_id': 'ibighit', 'average_likes': 1371766.25}
{'_id': 'melanie martinez', 'average_likes': 911871.5}
{'_id': 'BETNetworks', 'average_likes': 769687.625}
{'_id': 'jypentertainment', 'average_likes': 758826.6666666666}
{'_id': 'Logan Paul Vlogs', 'average_likes': 653443.5384615385}
{'_id': 'Wengie', 'average_likes': 567235.2}
{'_id': 'Alesso', 'average_likes': 509417.28571428574}
{'_id': 'PewDiePie', 'average_likes': 508647.8}
{'_id': 'shakiraVEVO', 'average_likes': 434302.14285714284}
{'_id': 'Ed Sheeran', 'average_likes': 395654.1666666667}
{'_id': 'Dude Perfect', 'average_likes': 389697.1111111111}
{'_id': 'Conor Maynard', 'average_likes': 389273.0}
{'_id': 'TaylorSwiftVEVO', 'average_likes': 367455.6666666667}
{'_id': 'Liza Koshy', 'average_likes': 365146.9583333333}
{'_id': 'Linkin Park', 'average_likes': 356475.7272727273}
{'_id': 'jbalvinVEVO', 'average_likes': 345130.4285714286}
{'_id': 'Primitive Technology', 'average_likes': 340